In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import timedelta
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')
folder_debris = os.path.join(folder_AGVA, 'debris cover', 'raster')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values

print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

3031


In [4]:
c = 0
for i in range(len(rgis_to_analyze)):
#     if c>0: continue
#     if i<43: continue
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
#     if rgiid!= "RGI60-01.15731": continue #10910 08989

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]

#     if ga<332: continue
    if ga<3000: continue
        
    # set folder
    if validation_only:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    else:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
       
    # check if this glacier has been run already, skip if so
    temp_path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_2022_daily_AAs_shadowed.nc")
    if os.path.exists(temp_path): continue
        
    # check if the initial products we need exist yet. if not, then continue
    path_1 = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_2022_daily_AAs.nc")
    path_2 = os.path.join(folder_save, 'Shadows', f"S2_{rgiid}_2022_daily_shadows.nc")
    if not os.path.exists(path_1): continue
    if not os.path.exists(path_2): continue
    
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry

    # single_geometry = single_geometry.buffer(-100) #what if we buffer out the exterior 100 meters of the glacier
 
    # open glacier mask, count how many pixels there are
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    
    # open the observed faction df
#     obs_path = os.path.join(folder_save, 'Daily AAs', 'observed', f"S2_{rgiid}_observed.csv")
#     obs_df = pd.read_csv(obs_path)
    
    # for each year, open the daily data, coarsen and resave
    for y in [2018,2019,2020,2021,2022]:
        if ga>100: print(y)
#         if y!=2018: continue
        
        # open data
        path_open = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs.nc")
        path_shadow = os.path.join(folder_save, 'Shadows', f"S2_{rgiid}_{y}_daily_shadows.nc")
        path_debris = os.path.join(folder_debris, f"{rgiid}_debris.tif")
        
        # if small glacier, we dont need to chunk
        if ga>1000:
            snow = xr.open_dataset(path_open, chunks={'time':1})
            shadows = xr.open_dataset(path_shadow, chunks={'time':1})
            debris = riox.open_rasterio(path_debris)
        elif ga>100:
            snow = xr.open_dataset(path_open, chunks={'time':10})
            shadows = xr.open_dataset(path_shadow, chunks={'time':10})
            debris = riox.open_rasterio(path_debris)
        else:
            snow = xr.open_dataset(path_open)
            shadows = xr.open_dataset(path_shadow)
            debris = riox.open_rasterio(path_debris)
        
        # at this point:
        # snow=0 is off-glacier, shadow, cloud
        # snow=1 is snow
        # snow=2,3,4,6 is firn,ice,debris,water (in that order) (5 is shadow)
        # terrain shadow mask has 1=usable, 0=shadow or off-glacier
        # debris mask has 1=debris, 0=not debris
        
        # apply debris mask. where 'snow' is not in terrain shadow and is classified as shadow, ice, or firn, but should be debris
        # make snow 4 where (debris=1) and (shadow=1) and (snow=0 or 2 or 3)
        snow = xr.where( (debris==1) & (shadows==1) & (snow.isin([2,3,5])), 4, snow)
        
#         # apply terrain shadow mask
#         # make snow 0 where shadows=0
        snow = snow.where(shadows==1, 0)
        
#         # make all shadows 0
        snow = snow.where(snow!=5,0)

#         # reapply mask
        snow = snow.where(glacier_mask>0.5,0).astype('uint8')
     
        # save
        path_temp = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed.nc")
        encoding = {"class":{"zlib": True, "dtype": "uint8"}}
        snow.to_netcdf(path_temp, encoding=encoding)
     
    c+=1
print("Done!")
# should start at 1953, 6.217 km2


Starting 3030 of 3031: RGI60-01.13635  3025.115 km2
2018
2019
2020
2021
2022

Starting 3031 of 3031: RGI60-01.13696  3362.656 km2
2018
2019
2020
2021
2022
Done!
